In [1]:
import cohere
import os
import time
import re
import pickle
import numpy as np
import json

In [3]:
# Ścieżka do folderu z plikami
dataset_path = "../../../dataset/datasets/MLPSD/final_dataset/final_test.pkl"

# Preprocessing dataset

In [4]:
# Wczytanie pliku
with open(dataset_path, 'rb') as f:
    data = pickle.load(f)

In [7]:
data

,annotation_id,annotator,created_at,errors_summary,id,lead_time,overall_summary,rating,squat_types,tricks,updated_at,video_url,fps,video_time,frames,labels,video_path,keypoints,distance_matrices_normalized,distance_matrices
0,13,1,2024-11-18T22:25:23.511525Z,NaN,366,698.367,NaN,"[{""number"":7}]",diffrent,"[{'start': 13.374851720047449, 'end': 15.34460...",2024-11-18T22:25:23.511525Z,s3://squat-label-studio/squats/lifting/0-0084-...,30.0,41.00,1230.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",squats/lifting/0-0084-50-7-0-2-0-0.mp4,None,None,None
1,11,1,2024-11-18T22:08:33.297645Z,NaN,367,267.864,NaN,"[{""number"":7}]",NaN,NaN,2024-11-18T22:09:00.052430Z,s3://squat-label-studio/squats/lifting/0-0085-...,30.0,14.00,420.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",squats/lifting/0-0085-80-8-1-2-0-0.mp4,None,None,None
2,12,1,2024-11-18T22:15:06.188387Z,NaN,368,413.600,Ćwiczenie zdecydowanie wykonywane zbyt szybko....,"[{""number"":6}]",NaN,"[{'start': 1.0688018979833926, 'end': 1.804863...",2024-12-05T15:41:23.188734Z,s3://squat-label-studio/squats/lifting/0-0086-...,30.0,17.00,510.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",squats/lifting/0-0086-1-5-0-3-0-0.mp4,None,None,None
3,28,1,2024-12-05T15:43:49.471138Z,NaN,369,142.490,NaN,"[{""number"":7}]",low_bar,NaN,2024-12-05T15:43:49.471138Z,s3://squat-label-studio/squats/lifting/0-0087-...,30.0,20.00,600.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",squats/lifting/0-0087-5-1-3-0-0.mp4,None,None,None
4,29,1,2024-12-05T15:44:43.549323Z,NaN,371,30.621,Ćwiczenie zdecydowanie wykonywane zbyt szybko....,"[{""number"":6}]",high_bar,NaN,2024-12-05T15:44:43.549323Z,s3://squat-label-studio/squats/lifting/0-0089-...,30.0,9.00,270.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",squats/lifting/0-0089-60-5-0-3-0-1.mp4,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,111,1,2024-12-06T12:46:14.276847Z,Przy każdym powtórzeniu zapadają się kolana do...,654,38.957,NaN,"[{""number"":1}]",low_bar,"[{'start': 1.2051113702290077, 'end': 2.856560...",2024-12-06T12:46:14.276847Z,s3://squat-label-studio/squats/single_individu...,29.0,10.02,298.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",squats/single_individual_dataset/0-162-1-3-0-1...,"[[[0.0, 0.0, 0.0], [0.09909302, -0.012729362, ...","[[0.038986996, 0.29564527, 0.53965086, 0.01586...","[[[0.0, 0.15708824, 0.5382082, 0.90053976, 0.1..."
150,110,1,2024-12-06T12:45:34.034476Z,Przy każdym powtórzeniu zapadają się kolana do...,655,54.978,NaN,"[{""number"":1}]",low_bar,"[{'start': 1.0648338562340967, 'end': 2.786421...",2024-12-06T12:45:34.034476Z,s3://squat-label-studio/squats/single_individu...,29.0,10.02,298.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",squats/single_individual_dataset/0-163-1-3-0-0...,"[[[0.0, 0.0, 0.0], [-0.14001301, 0.005449459, ...","[[0.032452863, 0.30428386, 0.536123, 0.0289129...","[[[0.0, 0.14833686, 0.53882176, 0.87185836, 0...."
151,109,1,2024-12-06T12:44:35.949422Z,Przy każdym powtórzeniu zapadają się kolana do...,656,47.978,NaN,"[{""number"":1}]",low_bar,"[{'start': 1.026576352417303, 'end': 2.8438077...",2024-12-06T12:44:35.949422Z,s3://squat-label-studio/squats/single_individu...,29.0,10.02,298.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",squats/single_individual_dataset/0-164-1-3-0-1...,"[[[0.0, 0.0, 0.0], [0.08751434, -0.06855369, 0...","[[0.009846405, 0.29173395, 0.5368007, 0.035145...","[[[0.0, 0.1111705, 0.53050834, 0.8950713, 0.14..."
152,108,1,2024-12-06T12:43:45.675659Z,Przy każdym powtórzeniu zapadają się kolana do...,657,58.553,NaN,"[{""number"":1}]",low_bar,"[{'start': 1.4601613956743005, 'end': 2.933075...",2024-12-06T12:43:45.675659Z,s3://squat-label-studio/squats/single_individu...,29.0,10.02,298.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",squats/single_individual_dataset/0-165-1-3-0-3...,"[[[0.0, 0.0, 0.0], [-0.10758543, 0.06399433, -...","[[0.033659752, 0.33959332, 0.55626667, 0.03

In [8]:
labels = data['labels']
answers = data['errors_summary']
evaluations = data['rating']
fps = data['fps']
filenames = data['video_url']
tricks = data['tricks']

labels_type = ["Squat-depth" ,
    "Back-round" ,
    "Taking-off-foot",
    "Knee-collapse" ,
    "Butt-wink" ,
    "Dominant-hip" ,
    "Lack-of-abdominal-tension",
    "No-knee-outlet",
    "Bad-head-movement",
    "Back-warped"
]

# Api

In [9]:
# Inicjalizacja klienta Cohere
# https://cohere.com/
# https://docs.cohere.com/reference/embed

with open('../api.txt', 'r', encoding='utf-8') as file:
    for line in file:
        if 'cohere:' in line:
            # Wyciągamy tekst po "cohere:"
            key_api = line.split('cohere:')[1].strip()
            
co = cohere.Client(key_api)

# Generowanie odpowiedzi

In [10]:
# Funkcja do generowania analizy techniki z komentarzy
def generate_technique_analysis_prompt(data, prompt):
    
    prompt += "Data: " + data[0]
    #prompt += f"\n Summary of technique:  {data[1]}"
    prompt += f"\n Overall evaluation:  {data[2]}"
    #prompt += f"\n fps of video: {data[3]}"
    
    response = co.generate(
        model='command',  # Poprawiony model
        prompt=prompt,
        max_tokens=300,
        temperature=0.7
    )
    print("Prompt: ", prompt)
    # Zwracamy wygenerowaną odpowiedź
    return response.generations[0].text

In [11]:
# Zbudowanie promptu na podstawie komentarzy
prompt = "You will get data about a recording in which there is a person squatting with a barbell and weight. Then you will get data for this recording, the first is an array where in each row there is an error and information about the time intervals in which the error occurred. Analyze this data and describe it for the user. Do not describe when the errors occurred, but rather what errors occurred and how many times. In the second section describe how the problems can be solved. List possible causes of their occurrence. Do not exceed 1000 characters. Avoid general answers like: 'The error is due to poor technique' "

In [12]:
analysis_comments_prompt = {}
# Ustawienie wyświetlania pełnych tablic
np.set_printoptions(threshold=np.inf)

# Wyświetlenie wyników
for i, filename in enumerate(filenames):
    start_time = time.time()

    print(f"Wyniki dla nagrania: {filename}, numer {i}")
    if i >= 10:
        break
        
    label = labels[i]
    answer = answers[i]
    evaluation = json.loads(evaluations[i])[0]["number"]
    fps_ = fps[i]
    trick = tricks[i]
    
    ## Łączenie labels z labels_type
    #combined_data = []
    #for label_row, label_type in zip(trick, labels_type):
    #    combined_data.append({"type": label_type, "values": label_row})
        
    result_string = ""
    if not isinstance(trick, float):
        result_string += f"\n\n"
        for item in trick:
            result_string += f"start: {round(item['start'],2)}, end: {round(item['end'],2)} : {item['labels']}\n\n"
            
    data_input = [result_string, answer, evaluation, fps_]
    
    # Generowanie analizy techniki, jeśli komentarze istnieją
    analysis = generate_technique_analysis_prompt(data_input, prompt)
  
    # Dodajemy wynik do listy
    analysis_comments_prompt[filename] = {
        'Analysis': analysis
    }
    
    print('*'*100)
    print(f"Analysis: \n {analysis}\n")
    print('*'*100)
    
    end_time = time.time()
    print(f"Przetwarzanie trwało {end_time - start_time:.4f} sekundy")
    
    print("-" * 40)

Wyniki dla nagrania: s3://squat-label-studio/squats/lifting/0-0084-50-7-0-2-0-0.mp4, numer 0
Prompt:  You will get data about a recording in which there is a person squatting with a barbell and weight. Then you will get data for this recording, the first is an array where in each row there is an error and information about the time intervals in which the error occurred. Analyze this data and describe it for the user. Do not describe when the errors occurred, but rather what errors occurred and how many times. In the second section describe how the problems can be solved. List possible causes of their occurrence. Do not exceed 1000 characters. Avoid general answers like: 'The error is due to poor technique' Data: 

start: 13.37, end: 15.34 : ['Dominant-hip']

start: 25.8, end: 27.31 : ['Dominant-hip']


 Overall evaluation:  7
****************************************************************************************************
Analysis: 
  From the provided information, here is a summary

In [13]:
data

,annotation_id,annotator,created_at,errors_summary,id,lead_time,overall_summary,rating,squat_types,tricks,updated_at,video_url,fps,video_time,frames,labels,video_path,keypoints,distance_matrices_normalized,distance_matrices
0,13,1,2024-11-18T22:25:23.511525Z,NaN,366,698.367,NaN,"[{""number"":7}]",diffrent,"[{'start': 13.374851720047449, 'end': 15.34460...",2024-11-18T22:25:23.511525Z,s3://squat-label-studio/squats/lifting/0-0084-...,30.0,41.00,1230.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",squats/lifting/0-0084-50-7-0-2-0-0.mp4,None,None,None
1,11,1,2024-11-18T22:08:33.297645Z,NaN,367,267.864,NaN,"[{""number"":7}]",NaN,NaN,2024-11-18T22:09:00.052430Z,s3://squat-label-studio/squats/lifting/0-0085-...,30.0,14.00,420.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",squats/lifting/0-0085-80-8-1-2-0-0.mp4,None,None,None
2,12,1,2024-11-18T22:15:06.188387Z,NaN,368,413.600,Ćwiczenie zdecydowanie wykonywane zbyt szybko....,"[{""number"":6}]",NaN,"[{'start': 1.0688018979833926, 'end': 1.804863...",2024-12-05T15:41:23.188734Z,s3://squat-label-studio/squats/lifting/0-0086-...,30.0,17.00,510.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",squats/lifting/0-0086-1-5-0-3-0-0.mp4,None,None,None
3,28,1,2024-12-05T15:43:49.471138Z,NaN,369,142.490,NaN,"[{""number"":7}]",low_bar,NaN,2024-12-05T15:43:49.471138Z,s3://squat-label-studio/squats/lifting/0-0087-...,30.0,20.00,600.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",squats/lifting/0-0087-5-1-3-0-0.mp4,None,None,None
4,29,1,2024-12-05T15:44:43.549323Z,NaN,371,30.621,Ćwiczenie zdecydowanie wykonywane zbyt szybko....,"[{""number"":6}]",high_bar,NaN,2024-12-05T15:44:43.549323Z,s3://squat-label-studio/squats/lifting/0-0089-...,30.0,9.00,270.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",squats/lifting/0-0089-60-5-0-3-0-1.mp4,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,111,1,2024-12-06T12:46:14.276847Z,Przy każdym powtórzeniu zapadają się kolana do...,654,38.957,NaN,"[{""number"":1}]",low_bar,"[{'start': 1.2051113702290077, 'end': 2.856560...",2024-12-06T12:46:14.276847Z,s3://squat-label-studio/squats/single_individu...,29.0,10.02,298.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",squats/single_individual_dataset/0-162-1-3-0-1...,"[[[0.0, 0.0, 0.0], [0.09909302, -0.012729362, ...","[[0.038986996, 0.29564527, 0.53965086, 0.01586...","[[[0.0, 0.15708824, 0.5382082, 0.90053976, 0.1..."
150,110,1,2024-12-06T12:45:34.034476Z,Przy każdym powtórzeniu zapadają się kolana do...,655,54.978,NaN,"[{""number"":1}]",low_bar,"[{'start': 1.0648338562340967, 'end': 2.786421...",2024-12-06T12:45:34.034476Z,s3://squat-label-studio/squats/single_individu...,29.0,10.02,298.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",squats/single_individual_dataset/0-163-1-3-0-0...,"[[[0.0, 0.0, 0.0], [-0.14001301, 0.005449459, ...","[[0.032452863, 0.30428386, 0.536123, 0.0289129...","[[[0.0, 0.14833686, 0.53882176, 0.87185836, 0...."
151,109,1,2024-12-06T12:44:35.949422Z,Przy każdym powtórzeniu zapadają się kolana do...,656,47.978,NaN,"[{""number"":1}]",low_bar,"[{'start': 1.026576352417303, 'end': 2.8438077...",2024-12-06T12:44:35.949422Z,s3://squat-label-studio/squats/single_individu...,29.0,10.02,298.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",squats/single_individual_dataset/0-164-1-3-0-1...,"[[[0.0, 0.0, 0.0], [0.08751434, -0.06855369, 0...","[[0.009846405, 0.29173395, 0.5368007, 0.035145...","[[[0.0, 0.1111705, 0.53050834, 0.8950713, 0.14..."
152,108,1,2024-12-06T12:43:45.675659Z,Przy każdym powtórzeniu zapadają się kolana do...,657,58.553,NaN,"[{""number"":1}]",low_bar,"[{'start': 1.4601613956743005, 'end': 2.933075...",2024-12-06T12:43:45.675659Z,s3://squat-label-studio/squats/single_individu...,29.0,10.02,298.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",squats/single_individual_dataset/0-165-1-3-0-3...,"[[[0.0, 0.0, 0.0], [-0.10758543, 0.06399433, -...","[[0.033659752, 0.33959332, 0.55626667, 0.03